# Step-by-step demo walk through

The testing demos are intended to be done in a code editor/IDE, and the tests run from the command line after each step with `pytest demo_tdd_intro.py`. Below each of the steps are laid out to guide the tutorial leader. I prefer to print this out so I can refer to it while I live code each step over a projector along with the attendees.

---

## TDD Demo

In [3]:
# initial code

def backwards_allcaps(text):
    return text[::-1].upper()


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'

In [4]:
test_backwards_allcaps()

### TDD task 1: whitespace should be removed from any input text

In [5]:
def backwards_allcaps(text):
    return text[::-1].replace(' ', '').upper()            # step 2


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'


def test_letters_only():
    assert backwards_allcaps('salt lake') == 'EKALTLAS'    # step 1

In [6]:
test_backwards_allcaps()
test_letters_only()

### TDD task 2: punctuation should also be removed

In [11]:
import string                                                          # step 2

def backwards_allcaps(text):
    for char in string.punctuation:                                    # step 2
        text = text.replace(char, '')                                  # step 2
    return text[::-1].replace(' ', '').upper()


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'


def test_letters_only():
    assert backwards_allcaps('salt lake') == 'EKALTLAS'
    assert backwards_allcaps('github.com is rad!') == 'DARSIMOCBUHTIG'  # step 1

In [12]:
test_backwards_allcaps()
test_letters_only()

### TDD task 3: passing an empty string should raise an error

In [13]:
import pytest                                                # step 2
import string

def backwards_allcaps(text):
    if len(text) == 0:                                       # step 2
        raise AttributeError('String must contain letters')  # step 2
    for char in string.punctuation:
        text = text.replace(char, '')
    return text[::-1].replace(' ', '').upper()


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'


def test_letters_only():
    assert backwards_allcaps('salt lake') == 'EKALTLAS'
    assert backwards_allcaps('github.com is rad!') == 'DARSIMOCBUHTIG'


def test_empty_string():                                     # step 1
    with pytest.raises(AttributeError):
        backwards_allcaps('')

In [14]:
test_backwards_allcaps()
test_letters_only()
test_empty_string()

### TDD task 3b: passing a string without letters should raise an error

In [23]:
import pytest
import string

def backwards_allcaps(text):
    text = text.replace(' ', '')                             # step 2 - move section
    for char in string.punctuation:
        text = text.replace(char, '')
    if len(text) == 0:                                       # step 2 - move section
        raise AttributeError('String must contain letters')
    return text[::-1].upper()


def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'


def test_letters_only():
    assert backwards_allcaps('salt lake') == 'EKALTLAS'
    assert backwards_allcaps('github.com is rad!') == 'DARSIMOCBUHTIG'


def test_empty_string():
    with pytest.raises(AttributeError):
        backwards_allcaps('')
    with pytest.raises(AttributeError):                     # step 1
        backwards_allcaps(';')                              # step 1
    with pytest.raises(AttributeError):                     # step 1
        backwards_allcaps(' ')                              # step 1

In [24]:
test_backwards_allcaps()
test_letters_only()
test_empty_string()

## Fixtures Demo
Most of the tests we have written use the same pattern of asserting that our function run on some input gives us an expected output. Instead of reusing this same assert pattern repeatedly we would like a way to avoid duplicating code. Additionally, each assert statement can be thought of as a unique test, but with the current set up pytest treats each 

can employ fixtures to parametrize a set of inputs and outputs.

In [ ]:
@pytest.fixture